In [140]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from functools import reduce


In [141]:
test_arr = []
for i in range(0,3):
    sub_i = []
    for j in range(0,3):
        sub_j = []
        for k in range(0,15):
            sub_k = []
            for l in range(0,5):
                sub_l = []
                for xx in range(0,5000):
                    sub_l.append(np.random.randint(2))
                sub_k.append(sub_l)
            sub_j.append(sub_k)
        sub_i.append(sub_j)
    test_arr.append(sub_i)

In [142]:
test_np_arr = np.asarray(test_arr)
print(test_np_arr.shape)

(3, 3, 15, 5, 5000)


In [5]:
normal_mul = np.empty((3,3,15))
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,15):
            normal_mul[i][j][k]=np.sum(reduce(lambda x,y:x*y,test_np_arr[i][j][k]))

In [44]:
outer_test_arr = np.empty((5,3,3,15,5000))
for l in range(0,5):
        for i in range(0,3):
            for j in range(0,3):
                for k in range(0,15):
                    outer_test_arr[l][i][j][k] = test_np_arr[i][j][k][l]

In [8]:
outer_mul = reduce(lambda x,y:x*y,outer_test_arr)
outer_sum = np.empty((3,3,15))
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,15):
            outer_sum[i][j][k] = np.sum(outer_mul[i][j][k])

In [9]:
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,15):
            if (normal_mul[i][j][k] != outer_sum[i][j][k]):
                print("I will kill myself.")

In [45]:
outer_gpu_arr = gpuarray.to_gpu(outer_test_arr.astype(np.float32))

In [211]:
%%time
a_tripled_cpu = []
for oo in range(0,100):
    #a_tripled = np_test_arr[0]*np_test_arr[1]*np_test_arr[2]*np_test_arr[3]
    a_tripled_cpu = reduce(lambda x,y:x*y,test_np_arr)

CPU times: user 472 ms, sys: 4 ms, total: 476 ms
Wall time: 474 ms


In [28]:
%%time
a_tripled_gpu = []
for oo in range(0,100):
    #a_tripled = (test_gpu_arr[0]*test_gpu_arr[1]*test_gpu_arr[2]*test_gpu_arr[3]).get()
    test_gpu_arr = gpuarray.to_gpu(outer_test_arr.astype(np.float32))
    a_tripled_gpu = (reduce(lambda x,y:x*y, test_gpu_arr)).get()
    a_summed = (reduce(lambda x,y:x+y,gpuarray.to_gpu(a_tripled_gpu.astype(np.float32)))).get()

KeyboardInterrupt: 

In [48]:
test_gpu_arr = gpuarray.to_gpu(outer_test_arr.astype(np.float32))

In [143]:
from pycuda.reduction import ReductionKernel

In [198]:
from pycuda.elementwise import ElementwiseKernel
map_mul_kernel = ElementwiseKernel("float *a, float *b,float *c",
                                  "c[i]=a[i]*b[i]",
                                  "map_mul_kernel")

In [199]:
sum_kernel = ReductionKernel(np.float32, neutral="0",
                      reduce_expr="a+b", map_expr="x[i]",
                      arguments="float *x")

In [205]:
a_gpu = gpuarray.to_gpu(np.array([1,2,3,4]).astype(np.float32))
c_gpu = gpuarray.empty_like(a_gpu)

map_mul_kernel(a_gpu,
               gpuarray.to_gpu(np.array([1,2,3,4]).astype(np.float32)),
              c_gpu)

In [206]:
c_gpu

array([ 1.,  4.,  9., 16.], dtype=float32)

In [163]:
reciever_np = np.empty((3,2)).astype(np.float32)
reciever_np=krnl(gpuarray.to_gpu(np.array([1,2,3]).astype(np.float32)))
reciever_np

array(6., dtype=float32)

In [41]:
a_tripled_gpu = (reduce(lambda x,y:x*y, test_gpu_arr)).get()

In [64]:
a_reduced_sum = krnl(test_gpu_arr[0][0][1],test_gpu_arr[1][0][1]).get()

In [137]:
a_reduced_sum = krnl(gpuarray.to_gpu(np.array([[0,1,2],[0,1,2]]).astype(np.float32)),
                     gpuarray.to_gpu(np.array([[0,1,2],[0,1,2]]).astype(np.float32))).get()

In [138]:
a_reduced_sum

array(10., dtype=float32)

In [67]:
%%time
for i in range(0,100):
    print(reduce(lambda x,y:x+y,map(lambda k,w:k*w,)))

1243.4

In [212]:
%%time
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,15):
            acc = reduce(lambda x,y:x+y,list(reduce(lambda k,w:k*w,test_np_arr[i][j][k])))
            #print(acc)
            #break

CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 112 ms


In [181]:
acc

182

In [211]:
%%time
simple_loop_gpu_arr = gpuarray.to_gpu(test_np_arr)
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,15):
            gpu_acc = gpuarray.to_gpu(np.ones((5000)))
            for l in range(0,5):
                map_mul_kernel(gpu_acc,
                                gpuarray.to_gpu(test_np_arr[i][j][k][l].astype(np.float32)),
                                gpu_acc)
                gpu_acc = gpuarray.to_gpu(gpu_acc)
            acc = sum_krnl(gpu_acc).get()
            #print(acc)

CPU times: user 184 ms, sys: 12 ms, total: 196 ms
Wall time: 197 ms


array(0., dtype=float32)

In [80]:
print(reduce(lambda x,y:x+y,list(reduce(lambda k,w:k*w,test_np_arr[0][0][0]))))

149


In [75]:
dd = map(lambda k,w:k*w,test_np_arr[0][0][0])

TypeError: 'map' object is not callable